# Тематическое моделирование обращений абонентов

In [36]:
import codecs
import numpy as np

Загружаем исходные данные

In [217]:
f = codecs.open("all_income.txt", "r", encoding="utf-8")
data=[]
line = f.readline()
data.append(line)
while line:
    line = f.readline()
    data.append(line)
f.close()

In [218]:
len(data)

211420

Определим глобальные частоты слов в обращениях, приведя их к нормальным формам. Поскольку 1000 обращений обрабатывается примерно за 3 минуты, ограничим выборку до 10000

In [220]:
mydict={}
data_train=data[0:10000]

In [221]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [222]:
for i in range (0,len(data_train)):
    if i % 1000 == 0:
        print i
    str1=data_train[i].split()
    for j in range (0,len(str1)):
        word1=str1[j]
        word1=morph.parse(word1)[0].normalized[0]
        #print word1
        if mydict.get(word1)>0:
            s=mydict.get(word1)
            dict2={word1:s+1}
            mydict.update(dict2)
        else:
                mydict.setdefault(word1,1)
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


Отбросим те слова, которые встречаются менее, чем в 1% обращений (т.е. менее 100 раз)

In [223]:
f2 = codecs.open("dict.txt", "w", encoding="utf-8")
for key in mydict.keys():
    s=mydict.get(key)
    if s>=100:
        f2.write(key+':'+str(s)+'\n')
f2.close()

In [224]:
f3 = codecs.open("dict_only_words.txt", "w", encoding="utf-8")
for key in mydict.keys():
    s=mydict.get(key)
    if s>=100:
        f3.write(key+'\n')
f3.close()

Из получившегося словаря в ручном режиме удалены слова, не определяющие тему обращения. Осталось 166 слов. Загрузим их в словарь

In [266]:
f4 = codecs.open("dict_only_words_filtered.txt", "r", encoding="utf-8")
dict_words_filtered={}
i=0
line = f4.readline()
dict_words_filtered.update({line:i})
while line:
    i+=1
    line = f4.readline()
    dict_words_filtered.update({line:i})
f4.close()

In [267]:
len(dict_words_filtered)

166

Сформируем выборку из 200 обращений, таких, что в них входит хотя бы одно слово из нашего словаря

In [290]:
dialog_word_matr=np.zeros((200,len(dict_words_filtered)))
data_train2=[]

In [309]:
i=0
n=0
while (i<200) and (n<1000):
    #if i % 1000 == 0:
    #    print i
    str1=data_train[n].split()
    m=0
    for j in range (0,len(str1)):
        word1=str1[j]
        word1=morph.parse(word1)[0].normalized[0]
        #print word1
        k=dict_words_filtered.get(word1+'\n')
        #if word1==u'роуминг':
        #    print data_train[n]
        #    print i
        #    print k
        if k>0:
            m=1
            dialog_word_matr[i][k]=1            
    if m==1:
        data_train2.append(data_train[n])
        i=i+1
    n+=1
    #print i

In [310]:
f5 = codecs.open("data_train2.txt", "w", encoding="utf-8")
for i in range(0,200):
    f5.write(data_train2[i])
f5.close()

Разметим вручную темы обращений:
1 - тариф
2 - перевод денежных средств
3 - подписка/контент
4 - проблемы со связью
5 - интернет/MMS
6 - услуги, подключенные на номере
7 - проверка баланса/разъяснение списаний
8 - перенос остатков
9 - зона покрытия
10 - восстановление SIM/коды PIN и PUK
11 - роуминг
12 - ответ по заявке
13 - черный список
14 - MNP
15 - обмен минут на гигабайты
16 - Теле2-тема
0 - другое

загрузим данные по темам обращений

In [311]:
f6 = codecs.open("data_themes.csv", "r", encoding="utf-8")
data_themes=np.zeros(200)
line = f6.readline()
i=0
data_themes[i]=int(line)
while line:
    line = f6.readline()
    i+=1
    if line:
        data_themes[i]=int(line)
f6.close()

исключим из матрицы частот слов и из вектора тем тему "другое"

In [312]:
dialog_word_matr_filtered=dialog_word_matr[[data_themes>np.zeros(200)]]
data_themes_filtered=data_themes[data_themes>np.zeros(200)]

In [313]:
len(data_themes_filtered)

164

In [314]:
num_themes=data_themes_filtered.max()

In [315]:
freq_word_theme_matr=np.zeros((len(dict_words_filtered),int(num_themes)))

In [316]:
num_themes

16.0

Заполним матрицу принадлежности слов к темам

In [317]:
for i in range (0,len(data_themes_filtered)):
    for j in range(0,len(dict_words_filtered)):
        if(dialog_word_matr_filtered[i][j])>0: 
            freq_word_theme_matr[j][int(data_themes_filtered[i])-1]+=1

Нормируем столбцы матрицы, чтобы не было перекоса в сторону тем обращений, которые чаще встречаются

In [318]:
freq_word_abs=freq_word_theme_matr.sum(axis=1)

In [319]:
freq_word_abs

array([ 0., 11.,  6.,  1.,  1.,  4.,  3.,  6.,  5.,  2.,  0.,  6.,  3.,
        3., 17.,  1.,  4.,  4.,  6.,  0.,  1.,  2.,  3.,  6.,  6.,  2.,
        1.,  1.,  2.,  2.,  7.,  2.,  8.,  4.,  0.,  2.,  5.,  1.,  2.,
        3.,  8., 13.,  5.,  5.,  0.,  4., 26.,  4.,  2.,  3.,  3.,  5.,
        2.,  2.,  3.,  8., 11.,  4.,  4.,  1., 21.,  6.,  4.,  6.,  2.,
        1., 21.,  1.,  1.,  2.,  2.,  4.,  1.,  3., 10.,  7.,  7.,  2.,
       27.,  9., 46.,  4.,  1.,  9., 22.,  5.,  4.,  3., 16.,  6., 11.,
        1., 12., 25.,  9.,  3.,  2.,  2.,  3.,  4.,  4.,  8.,  1.,  6.,
        7.,  1.,  1.,  4.,  3., 13.,  5.,  5.,  4.,  4.,  3.,  3.,  1.,
        2.,  5.,  1.,  3., 10.,  3.,  3., 10.,  7.,  0.,  6.,  3.,  3.,
        2.,  8.,  4.,  3.,  3.,  0.,  4.,  7.,  1.,  2.,  1.,  3., 11.,
       18.,  1.,  1., 12.,  3.,  4.,  2.,  2.,  7.,  1., 14.,  5.,  3.,
        3.,  3.,  3.,  2.,  9., 49.,  4., 16.,  4.,  6.])

In [320]:
freq_word_theme_normed=np.zeros((len(dict_words_filtered),int(num_themes)))
for i in range(0,len(dict_words_filtered)):
    for j in range(0,int(num_themes)):
        if freq_word_abs[i]>0: 
            freq_word_theme_normed[i][j]=(freq_word_theme_matr[i][j]+0.0)/freq_word_abs[i]

Создадим процедуру, которая будет по тексту определять вероятность его принадлежности к каждой из тем (индексы сдвинуты на 1 влево, т.к. исключена тема "другое")

In [321]:
def find_theme(test_str):
    prob_vect=np.zeros(int(num_themes))
    str1=test_str.split()
    for j in range (0,len(str1)):
        word1=str1[j]
        word1=morph.parse(word1)[0].normalized[0]
        #print word1
        k=dict_words_filtered.get(word1+'\n')
        #print k
        if k>0:
            prob_vect+=freq_word_theme_matr[k]
    return prob_vect/prob_vect.sum()

Тестируем....

In [322]:
find_theme(u'Здравствуйте Мой тариф  орион федеральный На каких условиях я могу использовать  сотовую связи свыше тарифного плана xxx xxx xx xx Здравствуйте! Мой тариф  орион федеральный На каких условиях я могу использовать  интернет свыше тарифного плана xxx xxx xx xx xxxxx xxxxx xxxxx xxxxx xxxxx xxxxx Да То есть сверх пакета интернет  бесплатный но скорость падает Всё вопросов нет Спасибо :)'
)

array([0.42738589, 0.        , 0.        , 0.02489627, 0.32365145,
       0.00829876, 0.04564315, 0.04564315, 0.02489627, 0.        ,
       0.09543568, 0.        , 0.        , 0.00414938, 0.        ,
       0.        ])

In [324]:
find_theme(u'Почему роуминг не подключается в Китае Проблема решена спасибо')

array([0.11538462, 0.        , 0.03846154, 0.03846154, 0.26923077,
       0.        , 0.07692308, 0.03846154, 0.        , 0.        ,
       0.26923077, 0.07692308, 0.        , 0.03846154, 0.        ,
       0.03846154])

In [325]:
find_theme(u'Здравствуйте! Алла Просьба подключить выход в Интернет для номера xxx xxx xx xx Да xxxxx xxxxx Спасибо До свидания')

array([0.17333333, 0.01333333, 0.06666667, 0.04      , 0.4       ,
       0.05333333, 0.05333333, 0.04      , 0.01333333, 0.        ,
       0.14666667, 0.        , 0.        , 0.        , 0.        ,
       0.        ])

Вывод: модель успешно определяет наиболее популярные темы обращений. Рекомендуется выполнить обучение модели на стратифицированной выборке большего объема, в этом случае можно ожидать существенное улучшение качества модели